# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,springbok,-29.6643,17.8865,14.13,13,0,5.04,ZA,1693167653
1,1,grytviken,-54.2811,-36.5092,-2.06,84,35,4.16,GS,1693167160
2,2,sainyabuli,19.2576,101.7103,22.29,98,100,0.28,LA,1693167654
3,3,urzhar,47.0930,81.6294,23.84,41,100,2.03,KZ,1693167654
4,4,iskateley,67.6803,53.1512,6.66,94,100,3.06,RU,1693167065


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
world_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    xlim = (-180,180),
    ylim = (-75,90),
    frame_width = 700,
    size = "Humidity",
    color = "City"
)

# Display the map
world_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filtered_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_filtered_df = city_data_filtered_df.dropna()

# Display sample data
city_data_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,n'dalatando,-9.2978,14.9116,23.25,60,0,2.66,AO,1693167679
103,103,ilha de mocambique,-15.0342,40.7358,24.84,83,0,4.08,MZ,1693167683
220,220,almeria,36.8381,-2.4597,26.41,66,0,2.06,ES,1693167497
246,246,baharly,38.4362,57.4316,23.31,33,0,2.37,TM,1693167496
256,256,quibala,-10.7337,14.9800,21.39,68,0,1.40,AO,1693167726
452,452,rosetta,31.3995,30.4172,26.69,74,0,3.61,EG,1693167780
477,477,imzouren,35.1464,-3.8506,24.60,38,0,2.57,MA,1693167787
509,509,grande prairie,55.1667,-118.8027,24.10,46,0,4.12,CA,1693167796
515,515,pampa del indio,-26.0498,-59.9373,21.24,17,0,4.22,AR,1693167798
531,531,maragogi,-9.0122,-35.2225,25.55,85,0,4.16,BR,1693167803


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
95,n'dalatando,AO,-9.2978,14.9116,60,
103,ilha de mocambique,MZ,-15.0342,40.7358,83,
220,almeria,ES,36.8381,-2.4597,66,
246,baharly,TM,38.4362,57.4316,33,
256,quibala,AO,-10.7337,14.9800,68,
452,rosetta,EG,31.3995,30.4172,74,
477,imzouren,MA,35.1464,-3.8506,38,
509,grande prairie,CA,55.1667,-118.8027,46,
515,pampa del indio,AR,-26.0498,-59.9373,17,
531,maragogi,BR,-9.0122,-35.2225,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    location_url = base_url + f"?categories={params['categories']}&filter={params['filter']}&bias={params['bias']}&limit={params['limit']}&apiKey={params['apiKey']}"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(location_url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
n'dalatando - nearest hotel: Hotel Terminus
ilha de mocambique - nearest hotel: Ruby Backpackers
almeria - nearest hotel: Hotel Costasol
baharly - nearest hotel: No hotel found
quibala - nearest hotel: No hotel found
rosetta - nearest hotel: No hotel found
imzouren - nearest hotel: Radisson Blu Resort,Al Hoceima
grande prairie - nearest hotel: Sandman Hotel Grande Prairie
pampa del indio - nearest hotel: Hotel La Casona
maragogi - nearest hotel: Pousada Mariluz
yorkton - nearest hotel: Ramada Yorkton


,City,Country,Lat,Lng,Humidity,Hotel Name
95,n'dalatando,AO,-9.2978,14.9116,60,Hotel Terminus
103,ilha de mocambique,MZ,-15.0342,40.7358,83,Ruby Backpackers
220,almeria,ES,36.8381,-2.4597,66,Hotel Costasol
246,baharly,TM,38.4362,57.4316,33,No hotel found
256,quibala,AO,-10.7337,14.9800,68,No hotel found
452,rosetta,EG,31.3995,30.4172,74,No hotel found
477,imzouren,MA,35.1464,-3.8506,38,"Radisson Blu Resort,Al Hoceima"
509,grande prairie,CA,55.1667,-118.8027,46,Sandman Hotel Grande Prairie
515,pampa del indio,AR,-26.0498,-59.9373,17,Hotel La Casona
531,maragogi,BR,-9.0122,-35.2225,85,Pousada Mariluz


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
ideal_locations_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    xlim = (-180,180),
    ylim = (-75,90),
    frame_width = 700,
    color = "City",
    hover_cols = "all"
)

# Display the map
ideal_locations_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,index,Country,Humidity,Hotel Name)